In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('Data/data_clean.csv')

col_target = 'Codigo'
col_features = [col for col in data.columns if col != col_target]

X = data[col_features]
y = data[col_target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Cargar los datos
data = pd.read_csv('Data/data_clean.csv')

# Paso 1: Codificación de la columna 'Codigo'
label_encoder = LabelEncoder()
data['Codigo_encoded'] = label_encoder.fit_transform(data['Codigo'])

# Paso 2: Tokenización y padding de la columna 'Descripcion'
max_words = 10000  # Número máximo de palabras que el tokenizador considerará
max_len = 100  # Longitud máxima de las secuencias

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['Descripcion'])

# Convertir las descripciones a secuencias de números
X_seq = tokenizer.texts_to_sequences(data['Descripcion'])

# Aplicar padding para que todas las secuencias tengan la misma longitud
X_pad = pad_sequences(X_seq, maxlen=max_len)

# Ahora X_pad es tu conjunto de características y 'Codigo_encoded' tu conjunto de etiquetas
X = X_pad
y = data['Codigo_encoded']

# División del conjunto en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Ahora ya puedes usar este dataset para entrenar el modelo


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

# Parámetros
embedding_dim = 64  # Dimensión de los embeddings (puedes ajustarla)

# Construcción del modelo
model = Sequential()

# Capa de Embedding: convierte palabras en vectores
model.add(Embedding(input_dim=10000, output_dim=embedding_dim))

# Capa de GlobalAveragePooling: reduce las dimensiones
model.add(GlobalAveragePooling1D())

# Primera capa densa
model.add(Dense(64, activation='relu'))

# Capa de salida: tantas neuronas como clases (etiquetas/códigos)
model.add(Dense(len(set(y_train)), activation='softmax'))

# Compilación del modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()


In [ ]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {test_acc:.4f}')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam

# Cargar el archivo CSV
data = pd.read_csv('Data/data_clean.csv')

# Definir las columnas objetivo y características
col_target = 'Codigo'
col_features = ['Descripcion']  # Nos centramos en la columna 'Descripcion'

X = data[col_features]
y = data[col_target]

# Paso 1: Codificación de las etiquetas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Paso 2: Tokenización y Padding de las descripciones
max_words = 10000  # Número máximo de palabras en el vocabulario del tokenizer
max_len = 100  # Longitud máxima de las secuencias

# Crear el tokenizador y ajustarlo en el texto de las descripciones
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X['Descripcion'])

# Convertir el texto en secuencias numéricas
X_seq = tokenizer.texts_to_sequences(X['Descripcion'])

# Aplicar padding para que todas las secuencias tengan la misma longitud
X_pad = pad_sequences(X_seq, maxlen=max_len)

# Paso 3: División del conjunto en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, stratify=y_encoded)

# Paso 4: Construcción del modelo
embedding_dim = 64  # Tamaño de los embeddings

model = Sequential()

# Capa de Embedding
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

# Capa GlobalAveragePooling1D para reducir la dimensionalidad
model.add(GlobalAveragePooling1D())

# Capa completamente conectada con 64 neuronas y activación ReLU
model.add(Dense(64, activation='relu'))

# Capa de salida con tantas neuronas como clases y activación softmax
model.add(Dense(len(set(y_encoded)), activation='softmax'))

# Compilación del modelo con el optimizador Adam y pérdida categórica
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()

# Paso 5: Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Paso 6: Evaluación del modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {test_acc:.4f}')
